In [1]:
import cv2 as cv
import torch
import numpy as np
import os
import json
from torch import nn
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.datasets import VOCDetection
from torchvision.transforms import ToTensor

from datasets import YoloVOCDataset
from utils.my_dataset import YoloDataset
from model import Yolov1

In [ ]:
device='cpu'
IMG_SIZE=224
S=7
C=20

grid_size=IMG_SIZE/S

model=Yolov1(S,C).to(device)
model.load_state_dict(torch.load('checkpoint.pth',map_location=device)['model'])
model.eval()

# ds=YoloVOCDataset(IMG_SIZE,S,C)

In [3]:
def draw_box(img,row,col,output):
    grid_size=IMG_SIZE/S

    # 坐标还原（仅在3x448x448中）
    cx,cy=grid_size*(col+output[0]),grid_size*(row+output[1])
    w,h=output[2]*IMG_SIZE,output[3]*IMG_SIZE
    xmin,ymin,xmax,ymax=cx-w/2,cy-h/2,cx+w/2,cy+h/2

    # 缩放到原图坐标
    x_scale=img.shape[1]/IMG_SIZE
    y_scale=img.shape[0]/IMG_SIZE
    cx,cy=cx*x_scale,cy*y_scale
    xmin,ymin,xmax,ymax=xmin*x_scale,ymin*y_scale,xmax*x_scale,ymax*y_scale

    # 画框rectangle
    cv.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)),color=(0,0,255))

In [6]:
# my_ds=YoloDataset(IMG_SIZE,S,C)
# img=my_ds.get_item(0)
# input=my_ds[0][0]
# img=cv.resize(img,(IMG_SIZE,IMG_SIZE))

img=cv.imread('data\VOCdevkit\VOC2012\JPEGImages\2007_000027.jpg')
img=cv.resize(img,(IMG_SIZE,IMG_SIZE))
input=ToTensor()()

output=model(input.unsqueeze(0))[0]

for i in range(S):
    for j in range(S):
        if output[i,j,4]>0.5:
            draw_box(img,i,j,output[i,j,:5])
        if output[i,j,9]>0.5:
            draw_box(img,i,j,output[i,j,5:10])
cv.imshow('Camera', img)
cv.waitKey(0)
cv.destroyAllWindows()


# cv.rectangle(img,(xmin,ymin),(xmax,ymax),(0,0,255))
# cv.imshow('img',img)
# cv.waitKey(0)
# cv.destroyAllWindows()

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [5]:
img

In [2]:
cv.imread('data\VOCdevkit\VOC2012\JPEGImages\2007_000027.jpg')